In [17]:
# Import dependencies
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()
from datetime import datetime, date

In [18]:
# Import configuration variables
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [19]:
# Set up simple_salesforce
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [20]:
# Connect to your MySQL database
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Query 'staffassignment' table from MySql

In [21]:
# Query Data
staff_sa = pd.read_sql("SELECT * FROM staffassignment", conn)
staff_sa.head()

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


## Query Class data from Salesforce in order to Get its 'Record Id'

In [22]:
sf_class_data = sf.query_all_iter("SELECT Id, Name, ID_Class__c FROM class__c")

In [23]:
class_sf = []

for data in sf_class_data:
    records = {
        'Class_Id_record': data['Id'],
        'Class Name' : data['Name'],
        'ID_Class__c': data['ID_Class__c']
    }
    
    class_sf.append(records)

In [24]:
class_sf

[{'Class_Id_record': 'a043h00000Lm9mxAAB',
  'Class Name': 'a043h00000Lm9mx',
  'ID_Class__c': 1.0},
 {'Class_Id_record': 'a043h00000Lm9n7AAB',
  'Class Name': 'a043h00000Lm9n7',
  'ID_Class__c': 1.0},
 {'Class_Id_record': 'a043h00000Lm9nCAAR',
  'Class Name': 'a043h00000Lm9nC',
  'ID_Class__c': 2.0},
 {'Class_Id_record': 'a043h00000Lm9nRAAR',
  'Class Name': 'a043h00000Lm9nR',
  'ID_Class__c': 3.0},
 {'Class_Id_record': 'a043h00000Lm9nbAAB',
  'Class Name': 'a043h00000Lm9nb',
  'ID_Class__c': 3.0},
 {'Class_Id_record': 'a043h00000Lm9ngAAB',
  'Class Name': 'a043h00000Lm9ng',
  'ID_Class__c': 4.0},
 {'Class_Id_record': 'a043h00000Lm9nWAAR',
  'Class Name': 'a043h00000Lm9nW',
  'ID_Class__c': 3.0},
 {'Class_Id_record': 'a043h00000Lm9nqAAB',
  'Class Name': 'a043h00000Lm9nq',
  'ID_Class__c': 4.0},
 {'Class_Id_record': 'a043h00000Lm9nMAAR',
  'Class Name': 'a043h00000Lm9nM',
  'ID_Class__c': 2.0},
 {'Class_Id_record': 'a043h00000Lm9n2AAB',
  'Class Name': 'a043h00000Lm9n2',
  'ID_Class__

In [25]:
class_sf = pd.DataFrame(class_sf)
class_sf.head()

,Class_Id_record,Class Name,ID_Class__c
0,a043h00000Lm9mxAAB,a043h00000Lm9mx,1.0
1,a043h00000Lm9n7AAB,a043h00000Lm9n7,1.0
2,a043h00000Lm9nCAAR,a043h00000Lm9nC,2.0
3,a043h00000Lm9nRAAR,a043h00000Lm9nR,3.0
4,a043h00000Lm9nbAAB,a043h00000Lm9nb,3.0


In [26]:
class_sf['ID_Class__c'] = class_sf['ID_Class__c'].astype(int)

## Prep. Staffassignment table to be prompt to join with Class table

In [27]:
# Transform Data to perfectly match between dataframe headers and field names in Salesforce
staff_sa = staff_sa.rename(columns={
                                        'ID_StaffAssignment':'ID_StaffAssignment__c',
                                        'ID_Staff':'ID_Staff__c',
                                        'ID_Class':'ID_Class__c',
                                        'Role':'Name',
                                        'StartDate':'StartDate__c',
                                        'EndDate':'EndDate__c'
                                })
staff_sa.head()

,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Name,StartDate__c,EndDate__c
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [28]:
'''
### doing this is still making an error of 'Object of type date is not JSON serializable' ###
staff_sa['StartDate__c'] = pd.to_datetime(staff_sa['StartDate__c']).dt.date
staff_sa['EndDate__c'] = pd.to_datetime(staff_sa['EndDate__c']).dt.date
'''

"\n### doing this is still making an error of 'Object of type date is not JSON serializable' ###\nstaff_sa['StartDate__c'] = pd.to_datetime(staff_sa['StartDate__c']).dt.date\nstaff_sa['EndDate__c'] = pd.to_datetime(staff_sa['EndDate__c']).dt.date\n"

In [29]:
from datetime import datetime, date
staff_sa['StartDate__c'] = staff_sa['StartDate__c'].apply(lambda x: x.strftime('%Y-%m-%d'))
staff_sa['EndDate__c'] = staff_sa['EndDate__c'].apply(lambda x: x.strftime('%Y-%m-%d')) 

AttributeError: 'NoneType' object has no attribute 'strftime'

## Join tables

In [30]:
staff_x_class = staff_sa.merge(class_sf)
staff_x_class.head()

,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Name,StartDate__c,EndDate__c,Class_Id_record,Class Name
0,1,1,1,Teacher Assistant,2020-03-16,None,a043h00000Lm9mxAAB,a043h00000Lm9mx
1,1,1,1,Teacher Assistant,2020-03-16,None,a043h00000Lm9n7AAB,a043h00000Lm9n7
2,1,1,1,Teacher Assistant,2020-03-16,None,a043h00000Lm9n2AAB,a043h00000Lm9n2
3,1,1,1,Teacher Assistant,2020-03-16,None,a043h00000Lm9shAAB,a043h00000Lm9sh
4,1,1,1,Teacher Assistant,2020-03-16,None,a043h00000Lm9sXAAR,a043h00000Lm9sX


In [31]:
# Convert DataFrame to a list of dictionaries --> This can help facilitate a bulk insert
staff_load = staff_x_class.to_dict(orient='record')
staff_load

[{'ID_StaffAssignment__c': 1,
  'ID_Staff__c': 1,
  'ID_Class__c': 1,
  'Name': 'Teacher Assistant',
  'StartDate__c': '2020-03-16',
  'EndDate__c': None,
  'Class_Id_record': 'a043h00000Lm9mxAAB',
  'Class Name': 'a043h00000Lm9mx'},
 {'ID_StaffAssignment__c': 1,
  'ID_Staff__c': 1,
  'ID_Class__c': 1,
  'Name': 'Teacher Assistant',
  'StartDate__c': '2020-03-16',
  'EndDate__c': None,
  'Class_Id_record': 'a043h00000Lm9n7AAB',
  'Class Name': 'a043h00000Lm9n7'},
 {'ID_StaffAssignment__c': 1,
  'ID_Staff__c': 1,
  'ID_Class__c': 1,
  'Name': 'Teacher Assistant',
  'StartDate__c': '2020-03-16',
  'EndDate__c': None,
  'Class_Id_record': 'a043h00000Lm9n2AAB',
  'Class Name': 'a043h00000Lm9n2'},
 {'ID_StaffAssignment__c': 1,
  'ID_Staff__c': 1,
  'ID_Class__c': 1,
  'Name': 'Teacher Assistant',
  'StartDate__c': '2020-03-16',
  'EndDate__c': None,
  'Class_Id_record': 'a043h00000Lm9shAAB',
  'Class Name': 'a043h00000Lm9sh'},
 {'ID_StaffAssignment__c': 1,
  'ID_Staff__c': 1,
  'ID_Class__c

## Insert 'Staffassignment' records

In [32]:
for row in staff_load:
    
    record = {
        'ID_StaffAssignment__c': row['ID_StaffAssignment__c'],
        'ID_Staff__c': row['ID_Staff__c'],
        'ID_Class__c':row['ID_Class__c'],
        'Name' : row['Name'],
        'StartDate__c':row['StartDate__c'],
        'EndDate__c':row['EndDate__c']
    }
    try:
        sf.Staff_Assignment__c.create(record)
    except Exception as e:
        print(e)

In [ ]:
'''
# Inserting rows in bulk for Staff Assignment table on Salesforce
try:
    sf.bulk.Staff_Assignment__c.insert(staff_load)
except Exception as e:
    print(e)
'''